# Tunability for DecisionTreeClassifier

# Intro

In [21]:
import pandas as pd

random_state = pd.read_csv("../../data/vars.csv")['random_state'].iloc[0]
n_iter = pd.read_csv("../../data/vars.csv")['n_iter'].iloc[0]
n_iter_BS = pd.read_csv("../../data/vars.csv")['n_iter_BS'].iloc[0]

In [2]:
df_college = pd.read_csv("../../data/processed/college.csv")
df_credit = pd.read_csv("../../data/processed/credit.csv")
df_diabetes = pd.read_csv("../../data/processed/diabetes.csv")
df_penguins = pd.read_csv("../../data/processed/penguins.csv")

In [3]:
print(df_college.info())
print()
print(df_credit.info())
print()
print(df_diabetes.info())
print()
print(df_penguins.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   type_school            1000 non-null   object 
 1   school_accreditation   1000 non-null   object 
 2   gender                 1000 non-null   object 
 3   interest               1000 non-null   object 
 4   residence              1000 non-null   object 
 5   parent_age             1000 non-null   int64  
 6   parent_salary          1000 non-null   int64  
 7   house_area             1000 non-null   float64
 8   average_grades         1000 non-null   float64
 9   parent_was_in_college  1000 non-null   bool   
 10  will_go_to_college     1000 non-null   int64  
dtypes: bool(1), float64(2), int64(3), object(5)
memory usage: 79.2+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
import numpy as np


num_pipeline = Pipeline(steps=[
    ('num_impute', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler())
])

cat_pipeline = Pipeline(steps=[
    ('cat_impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OneHotEncoder(handle_unknown='ignore'))
])

col_trans = ColumnTransformer([
    ('num_pipeline', num_pipeline, make_column_selector(dtype_include = np.number)),
    ('cat_pipeline', cat_pipeline, make_column_selector(dtype_include = np.object_))
])

model_pipeline = Pipeline(steps=[
    ('preprocessing', col_trans),
    ('model', DecisionTreeClassifier())
])

# RandomizedSearchCV

In [5]:
tree_params = {
    'model__criterion' : ['gini', 'entropy', 'log_loss'],
    'model__max_depth': range(2, 20),
    'model__min_samples_split': range(2, 21),                  
    'model__min_samples_leaf': range(1, 21),                         
    'model__max_leaf_nodes': range(10, 100, 10)
}

## Credit dataset

In [6]:
from sklearn.model_selection import RandomizedSearchCV

X = df_credit.iloc[:, 0:-1]
y = df_credit.iloc[:, -1]

random_search_model = RandomizedSearchCV(estimator = model_pipeline,
                                   param_distributions = tree_params,
                                   n_iter = n_iter,
                                   cv = 5,
                                   scoring = 'f1',
                                   random_state = random_state)

random_search_model.fit(X, y)

best_f1_credit = random_search_model.best_score_

print(f"Best f1 score: {best_f1_credit}")
print(f"Best params: {random_search_model.best_params_}")

random_search_model.best_estimator_

Best f1 score: 0.8648002281347171
Best params: {'model__min_samples_split': 17, 'model__min_samples_leaf': 7, 'model__max_leaf_nodes': 40, 'model__max_depth': 2, 'model__criterion': 'log_loss'}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BFFA39C790>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BFFA382110>)])),
                ('model',
                 DecisionTreeClassifier(criterion='log_loss', max_depth=2,
                                        max_leaf_nodes=40, min_samples_leaf=7,
                                        min_samples_split=17))])

## Data frame with params, results and mean result

In [7]:
tested_params = random_search_model.cv_results_['params']

In [8]:
df_decision_tree = pd.DataFrame(tested_params)
df_decision_tree['credit_f1'] = random_search_model.cv_results_['mean_test_score']
df_decision_tree

,model__min_samples_split,model__min_samples_leaf,model__max_leaf_nodes,model__max_depth,model__criterion,credit_f1
0,3,4,60,13,log_loss,0.789143
1,12,2,40,18,entropy,0.805944
2,16,8,10,14,gini,0.828757
3,4,9,40,3,gini,0.796802
4,10,14,10,14,gini,0.824301
...,...,...,...,...,...,...
95,10,14,80,19,log_loss,0.828465
96,18,8,80,16,entropy,0.802634
97,18,7,70,3,gini,0.797871
98,15,16,90,8,gini,0.842323


## College dataset

In [9]:
from sklearn.model_selection import GridSearchCV

X = df_college.iloc[:, 0:-1]
y = df_college.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_decision_tree['college_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_college = grid_search_model.best_score_

print(f"Best f1 score: {best_f1_college}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

Best f1 score: 0.8638314055176395
Best params: {'model__criterion': 'entropy', 'model__max_depth': 19, 'model__max_leaf_nodes': 20, 'model__min_samples_leaf': 17, 'model__min_samples_split': 11}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BFFC613250>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BFFC3006D0>)])),
                ('model',
                 DecisionTreeClassifier(criterion='entropy', max_depth=19,
                                        max_leaf_nodes=20, min_samples_leaf=17,
                                        min_samples_split=11))])

## Diabetes dataset

In [10]:
from sklearn.model_selection import GridSearchCV

X = df_diabetes.iloc[:, 0:-1]
y = df_diabetes.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_decision_tree['diabetes_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_diabetes = grid_search_model.best_score_

print(f"Best f1 score: {best_f1_diabetes}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

Best f1 score: 0.6450258878297515
Best params: {'model__criterion': 'gini', 'model__max_depth': 7, 'model__max_leaf_nodes': 30, 'model__min_samples_leaf': 18, 'model__min_samples_split': 10}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BFFC300690>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BFFD66D510>)])),
                ('model',
                 DecisionTreeClassifier(max_depth=7, max_leaf_nodes=30,
                                        min_samples_leaf=18,
                                        min_samples_split=10))])

## Penguins dataset

In [11]:
from sklearn.model_selection import GridSearchCV

X = df_penguins.iloc[:, 0:-1]
y = df_penguins.iloc[:, -1]

formatted_params = []
for params in tested_params:
    formatted_param = {key: [value] for key, value in params.items()}
    formatted_params.append(formatted_param)

grid_search_model = GridSearchCV(model_pipeline,
                                 param_grid=formatted_params,
                                 scoring='f1',
                                 cv = 5)

grid_search_model.fit(X, y)

df_decision_tree['penguins_f1'] = grid_search_model.cv_results_['mean_test_score']

best_f1_penguins = grid_search_model.best_score_


print(f"Best f1 score: {best_f1_penguins}")
print(f"Best params: {grid_search_model.best_params_}")
grid_search_model.best_estimator_

Best f1 score: 0.9967213114754099
Best params: {'model__criterion': 'log_loss', 'model__max_depth': 8, 'model__max_leaf_nodes': 50, 'model__min_samples_leaf': 1, 'model__min_samples_split': 16}


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BFFC315210>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BFFC50E110>)])),
                ('model',
                 DecisionTreeClassifier(criterion='log_loss', max_depth=8,
                                        max_leaf_nodes=50,
                                        min_samples_split=16))])

## Tunability 

In [12]:
df_decision_tree['mean_f1'] = df_decision_tree[['credit_f1', 'college_f1', 'diabetes_f1', 'penguins_f1']].mean(axis=1)
df_decision_tree

,model__min_samples_split,model__min_samples_leaf,model__max_leaf_nodes,model__max_depth,model__criterion,credit_f1,college_f1,diabetes_f1,penguins_f1,mean_f1
0,3,4,60,13,log_loss,0.789143,0.837255,0.564874,0.989944,0.795304
1,12,2,40,18,entropy,0.805944,0.836790,0.560251,0.982807,0.796448
2,16,8,10,14,gini,0.828757,0.845299,0.612650,0.986554,0.818315
3,4,9,40,3,gini,0.796802,0.804446,0.513266,0.986554,0.775267
4,10,14,10,14,gini,0.824301,0.846368,0.613375,0.986554,0.817649
...,...,...,...,...,...,...,...,...,...,...
95,10,14,80,19,log_loss,0.828465,0.854927,0.588572,0.986554,0.814629
96,18,8,80,16,entropy,0.802634,0.839371,0.584711,0.986554,0.803317
97,18,7,70,3,gini,0.797871,0.804743,0.513266,0.986554,0.775608
98,15,16,90,8,gini,0.842323,0.858438,0.614440,0.986554,0.825439


In [13]:
# sigma star
sigma_star_row = df_decision_tree[df_decision_tree['mean_f1'] == max(df_decision_tree['mean_f1'])]
sigma_star_row

,model__min_samples_split,model__min_samples_leaf,model__max_leaf_nodes,model__max_depth,model__criterion,credit_f1,college_f1,diabetes_f1,penguins_f1,mean_f1
76,11,17,20,19,entropy,0.845162,0.863831,0.624466,0.986554,0.830003


### (3.3) Measuring Overall Tunability of a ML Algorithm

In [14]:
d_credit = sigma_star_row['credit_f1'].iloc[0] - best_f1_credit
d_college = sigma_star_row['college_f1'].iloc[0] - best_f1_college
d_diabetes = sigma_star_row['diabetes_f1'].iloc[0] - best_f1_diabetes
d_penguins = sigma_star_row['penguins_f1'].iloc[0] - best_f1_penguins

d = pd.Series([d_credit, d_college, d_diabetes, d_penguins])
print(d)
print(f"Mean: {d.mean()}")


0   -0.019638
1    0.000000
2   -0.020560
3   -0.010168
dtype: float64
Mean: -0.012591389683811127


# BayesSearchCV

In [18]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

tree_params = {
    'model__criterion' : Categorical(['gini', 'entropy', 'log_loss']),
    'model__max_depth': Integer(2, 20),
    'model__min_samples_split': Integer(2, 21),                  
    'model__min_samples_leaf': Integer(1, 21),                         
    'model__max_leaf_nodes': Integer(10, 100)
}

## Credit dataset

In [27]:
X = df_credit.iloc[:, 0:-1]
y = df_credit.iloc[:, -1]

bayes_search_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=tree_params,
                                n_iter=int(n_iter_BS),
                                random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_model.fit(X, y)

best_BS_f1_credit = bayes_search_model.best_score_

print(f"Best f1 score: {best_BS_f1_credit}")
print(f"Best params: {bayes_search_model.best_params_}")
bayes_search_model.best_estimator_

Best f1 score: 0.8648002281347171
Best params: OrderedDict([('model__criterion', 'gini'), ('model__max_depth', 2), ('model__max_leaf_nodes', 65), ('model__min_samples_leaf', 21), ('model__min_samples_split', 17)])


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BFFDBA7550>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BFFEC420D0>)])),
                ('model',
                 DecisionTreeClassifier(max_depth=2, max_leaf_nodes=65,
                                        min_samples_leaf=21,
                                        min_samples_split=17))])

## College dataset

In [29]:
X = df_college.iloc[:, 0:-1]
y = df_college.iloc[:, -1]

bayes_search_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=tree_params,
                                n_iter=int(n_iter_BS),
                                random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_model.fit(X, y)

best_BS_f1_college = bayes_search_model.best_score_

print(f"Best f1 score: {best_BS_f1_college}")
print(f"Best params: {bayes_search_model.best_params_}")
bayes_search_model.best_estimator_

Best f1 score: 0.8627268515134437
Best params: OrderedDict([('model__criterion', 'entropy'), ('model__max_depth', 10), ('model__max_leaf_nodes', 93), ('model__min_samples_leaf', 16), ('model__min_samples_split', 11)])


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BF80E062D0>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BF80CDA0D0>)])),
                ('model',
                 DecisionTreeClassifier(criterion='entropy', max_depth=10,
                                        max_leaf_nodes=93, min_samples_leaf=16,
                                        min_samples_split=11))])

## Diabetes dataset

In [30]:
X = df_diabetes.iloc[:, 0:-1]
y = df_diabetes.iloc[:, -1]

bayes_search_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=tree_params,
                                n_iter=int(n_iter_BS),
                                random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_model.fit(X, y)

best_BS_f1_diabetes = bayes_search_model.best_score_

print(f"Best f1 score: {best_BS_f1_diabetes}")
print(f"Best params: {bayes_search_model.best_params_}")
bayes_search_model.best_estimator_

Best f1 score: 0.6417007553711286
Best params: OrderedDict([('model__criterion', 'gini'), ('model__max_depth', 11), ('model__max_leaf_nodes', 100), ('model__min_samples_leaf', 19), ('model__min_samples_split', 21)])


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BF80DDBF90>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BF8108C090>)])),
                ('model',
                 DecisionTreeClassifier(max_depth=11, max_leaf_nodes=100,
                                        min_samples_leaf=19,
                                        min_samples_split=21))])

## Penguins dataset

In [31]:
X = df_penguins.iloc[:, 0:-1]
y = df_penguins.iloc[:, -1]

bayes_search_model = BayesSearchCV(estimator=model_pipeline,
                                search_spaces=tree_params,
                                n_iter=int(n_iter_BS),
                                random_state=random_state,
                                cv=5,
                                scoring='f1')
                                
bayes_search_model.fit(X, y)

best_BS_f1_penguins = bayes_search_model.best_score_

print(f"Best f1 score: {best_BS_f1_penguins}")
print(f"Best params: {bayes_search_model.best_params_}")
bayes_search_model.best_estimator_

Best f1 score: 0.9967213114754099
Best params: OrderedDict([('model__criterion', 'log_loss'), ('model__max_depth', 4), ('model__max_leaf_nodes', 20), ('model__min_samples_leaf', 1), ('model__min_samples_split', 2)])


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipeline',
                                                  Pipeline(steps=[('num_impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BFFDE967D0>),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('cat_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001BFFDBA74D0>)])),
                ('model',
                 DecisionTreeClassifier(criterion='log_loss', max_depth=4,
                                        max_leaf_nodes=20))])

## Tunability

### (3.3) Measuring Overall Tunability of a ML Algorithm

In [33]:
d_BS_credit = sigma_star_row['credit_f1'].iloc[0] - best_BS_f1_credit
d_BS_college = sigma_star_row['college_f1'].iloc[0] - best_BS_f1_college
d_BS_diabetes = sigma_star_row['diabetes_f1'].iloc[0] - best_BS_f1_diabetes
d_BS_penguins = sigma_star_row['penguins_f1'].iloc[0] - best_BS_f1_penguins

d_BS = pd.Series([d_BS_credit, d_BS_college, d_BS_diabetes, d_BS_penguins])
print(d_BS)
print(f"Mean: {d_BS.mean()}")

0   -0.019638
1    0.001105
2   -0.017235
3   -0.010168
dtype: float64
Mean: -0.011483968068106476
